In [789]:
from pyspark.sql import SparkSession

In [790]:
#Mozemy pobawic sie interfejsem uzytkownika
#from pyspark import SparkContext
#sc = SparkContext(master = 'local[2]')
#sc

In [791]:
spark = SparkSession.builder.appName("MLwithSpark").getOrCreate()

### Firma Analit specjalizujaca sie w analizach zlecila mi opracowanie modelu szybkiego wykrywania chorob wartoby u pacjentow

In [792]:
# Predykcja czy pacjent jest chory na WZW typu C czy nie

In [793]:
df_0 = spark.read.csv("dane_hcvcsv", header = True)

In [794]:
df_0.show()

+---+-------------+---+---+----+----+----+----+----+-----+----+----+----+----+
|_c0|     Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL|CREA| GGT|PROT|
+---+-------------+---+---+----+----+----+----+----+-----+----+----+----+----+
|  1|0=Blood Donor| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23| 106|12.1|  69|
|  2|0=Blood Donor| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8|  74|15.6|76.5|
|  3|0=Blood Donor| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2|  86|33.2|79.3|
|  4|0=Blood Donor| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74|  80|33.8|75.7|
|  5|0=Blood Donor| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32|  76|29.9|68.7|
|  6|0=Blood Donor| 32|  m|41.6|43.3|18.5|19.7|12.3| 9.92|6.05| 111|  91|  74|
|  7|0=Blood Donor| 32|  m|46.3|41.3|17.5|17.8| 8.5| 7.01|4.79|  70|16.9|74.5|
|  8|0=Blood Donor| 32|  m|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6| 109|21.5|67.1|
|  9|0=Blood Donor| 32|  m|50.9|65.5|23.2|21.2| 6.9| 8.69| 4.1|  83|13.7|71.3|
| 10|0=Blood Donor| 32|  m|42.4|86.3|20.3|  20|35.2|

22/01/21 18:15:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Category, Age, Sex, ALB, ALP, ALT, AST, BIL, CHE, CHOL, CREA, GGT, PROT
 Schema: _c0, Category, Age, Sex, ALB, ALP, ALT, AST, BIL, CHE, CHOL, CREA, GGT, PROT
Expected: _c0 but found: 
CSV file: file:///home/student/dane_hcvcsv


In [795]:
print(df_0.columns)

['_c0', 'Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']


In [796]:
# Pozbywamy sie unikalnego id pacjenta

In [797]:
df_0 = df_0.select('Category','Age', 'Sex', 'ALB', 'ALP','ALT', 'AST', 'BIL', 'CHE', 'CHOL','CREA','GGT','PROT')

In [798]:
df_0.show(10)

+-------------+---+---+----+----+----+----+----+-----+----+----+----+----+
|     Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL|CREA| GGT|PROT|
+-------------+---+---+----+----+----+----+----+-----+----+----+----+----+
|0=Blood Donor| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23| 106|12.1|  69|
|0=Blood Donor| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8|  74|15.6|76.5|
|0=Blood Donor| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2|  86|33.2|79.3|
|0=Blood Donor| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74|  80|33.8|75.7|
|0=Blood Donor| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32|  76|29.9|68.7|
|0=Blood Donor| 32|  m|41.6|43.3|18.5|19.7|12.3| 9.92|6.05| 111|  91|  74|
|0=Blood Donor| 32|  m|46.3|41.3|17.5|17.8| 8.5| 7.01|4.79|  70|16.9|74.5|
|0=Blood Donor| 32|  m|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6| 109|21.5|67.1|
|0=Blood Donor| 32|  m|50.9|65.5|23.2|21.2| 6.9| 8.69| 4.1|  83|13.7|71.3|
|0=Blood Donor| 32|  m|42.4|86.3|20.3|  20|35.2| 5.46|4.45|  81|15.9|69.9|
+-------------+---+---+--

In [799]:
#sprawdzam jak zbalansowany jest zbior danych w zaleznosci od kategorii

In [800]:
df_0.groupBy("Category").count().show()

+--------------------+-----+
|            Category|count|
+--------------------+-----+
|       0=Blood Donor|  533|
|         3=Cirrhosis|   30|
|          2=Fibrosis|   21|
|0s=suspect Blood ...|    7|
|         1=Hepatitis|   24|
+--------------------+-----+



In [801]:
# Najwiecej jest zdrowych dawcow

In [802]:
# Teraz zanim zaczne zmianiac typy danych i obliczac wektory sprawdzam ile mam wartosci null i w zaleznosci od tego co mozna z nimi zrobic

In [803]:
from pyspark.sql.functions import isnan, when, count, col

df_0.select([count(when(isnan(c), c)).alias(c) for c in df_0.columns]).show()

+--------+---+---+---+---+---+---+---+---+----+----+---+----+
|Category|Age|Sex|ALB|ALP|ALT|AST|BIL|CHE|CHOL|CREA|GGT|PROT|
+--------+---+---+---+---+---+---+---+---+----+----+---+----+
|       0|  0|  0|  0|  0|  0|  0|  0|  0|   0|   0|  0|   0|
+--------+---+---+---+---+---+---+---+---+----+----+---+----+



In [804]:
# Nie mam wartosci null jako takich, ale jeszcze zamienie wartosci NA na 0

In [805]:
df_ = df_0.toPandas().replace('NA',"0")

In [806]:
df = spark.createDataFrame(df_)

In [807]:
#sprawdzam typy danych
df.dtypes

[('Category', 'string'),
 ('Age', 'string'),
 ('Sex', 'string'),
 ('ALB', 'string'),
 ('ALP', 'string'),
 ('ALT', 'string'),
 ('AST', 'string'),
 ('BIL', 'string'),
 ('CHE', 'string'),
 ('CHOL', 'string'),
 ('CREA', 'string'),
 ('GGT', 'string'),
 ('PROT', 'string')]

#### Wszystkie dane sa niestety stringami, lacznie z poszczegolnymi stezeniami. Musze zamienic kategorie oraz plec na int, a reszte na float

#### Znajduje unikalne wartosci kolumn Category i Sex

In [808]:
df.select('Category').distinct().collect()

[Row(Category='0=Blood Donor'),
 Row(Category='3=Cirrhosis'),
 Row(Category='2=Fibrosis'),
 Row(Category='0s=suspect Blood Donor'),
 Row(Category='1=Hepatitis')]

In [809]:
df.select('Sex').distinct().collect()

[Row(Sex='m'), Row(Sex='f')]

#### Nastepnie zamieniam na cyfry w nastepujacy sposob:
a) 'Category'
+ 0 - zdrowy dawca krwi
+ 1 - podejrzany dawca krwi
+ 2 - chory na zapalenie watroby
+ 3 - chory na zwloknieine watroby
+ 4 - chory na marskosc watroby

b) 'Sex'
+ 0 - m (mezczyzna)
+ 1 - f (kobieta)

In [810]:
df1 = df.toPandas().replace('0=Blood Donor',0)\
.replace('0s=suspect Blood Donor',1)\
.replace('1=Hepatitis',2)\
.replace('2=Fibrosis',3)\
.replace('3=Cirrhosis',4)

In [811]:
# konwersja z powrotem do dataframe Sparka

In [812]:
df_a = spark.createDataFrame(df1)

In [813]:
df_a.show(5)

+--------+---+---+----+----+----+----+----+-----+----+----+----+----+
|Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL|CREA| GGT|PROT|
+--------+---+---+----+----+----+----+----+-----+----+----+----+----+
|       0| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23| 106|12.1|  69|
|       0| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8|  74|15.6|76.5|
|       0| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2|  86|33.2|79.3|
|       0| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74|  80|33.8|75.7|
|       0| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32|  76|29.9|68.7|
+--------+---+---+----+----+----+----+----+-----+----+----+----+----+
only showing top 5 rows



In [814]:
#### teraz plec

In [815]:
df_a =df_a.toPandas().replace('m',0).replace('f',1)

In [816]:
df_b = spark.createDataFrame(df_a)

In [817]:
#sprawdzam znow typy danych
df_b.dtypes

[('Category', 'bigint'),
 ('Age', 'string'),
 ('Sex', 'bigint'),
 ('ALB', 'string'),
 ('ALP', 'string'),
 ('ALT', 'string'),
 ('AST', 'string'),
 ('BIL', 'string'),
 ('CHE', 'string'),
 ('CHOL', 'string'),
 ('CREA', 'string'),
 ('GGT', 'string'),
 ('PROT', 'string')]

In [818]:
#Zamieniam nastepnie kolumny ze stezeniami na float, wiek na int

In [819]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType , IntegerType

df_b = df_b.withColumn("Age",col("Age").cast(IntegerType()))\
.withColumn("ALB",col("ALB").cast(DoubleType()))\
.withColumn("ALP",col("ALP").cast(DoubleType()))\
.withColumn("ALT",col("ALT").cast(DoubleType()))\
.withColumn("AST",col("AST").cast(DoubleType()))\
.withColumn("BIL",col("BIL").cast(DoubleType()))\
.withColumn("CHE",col("CHE").cast(DoubleType()))\
.withColumn("CHOL",col("CHOL").cast(DoubleType()))\
.withColumn("CREA",col("CREA").cast(DoubleType()))\
.withColumn("GGT",col("GGT").cast(DoubleType()))\
.withColumn("PROT",col("PROT").cast(DoubleType()))

In [820]:
df_b.dtypes

[('Category', 'bigint'),
 ('Age', 'int'),
 ('Sex', 'bigint'),
 ('ALB', 'double'),
 ('ALP', 'double'),
 ('ALT', 'double'),
 ('AST', 'double'),
 ('BIL', 'double'),
 ('CHE', 'double'),
 ('CHOL', 'double'),
 ('CREA', 'double'),
 ('GGT', 'double'),
 ('PROT', 'double')]

In [821]:
#Wektoryzacja

In [822]:
required_columns = ["Category","Age","Sex","ALB","ALP","ALT","AST","BIL","CHE","CHOL","CREA","GGT","PROT"]

In [823]:
from pyspark.ml.feature import VectorAssembler

In [824]:
#df_final = v.transform(vec).setHandleInvalid("skip").transform(vec)

In [825]:
v = VectorAssembler(inputCols=required_columns,outputCol = 'vectors')

In [826]:
vec = v.transform(df_b)

In [827]:
vec.show(10)

+--------+---+---+----+----+----+----+----+-----+----+-----+----+----+--------------------+
|Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|             vectors|
+--------+---+---+----+----+----+----+----+-----+----+-----+----+----+--------------------+
|       0| 32|  0|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|69.0|[0.0,32.0,0.0,38....|
|       0| 32|  0|38.5|70.3|18.0|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|[0.0,32.0,0.0,38....|
|       0| 32|  0|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|[0.0,32.0,0.0,46....|
|       0| 32|  0|43.2|52.0|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|[0.0,32.0,0.0,43....|
|       0| 32|  0|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|[0.0,32.0,0.0,39....|
|       0| 32|  0|41.6|43.3|18.5|19.7|12.3| 9.92|6.05|111.0|91.0|74.0|[0.0,32.0,0.0,41....|
|       0| 32|  0|46.3|41.3|17.5|17.8| 8.5| 7.01|4.79| 70.0|16.9|74.5|[0.0,32.0,0.0,46....|
|       0| 32|  0|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|[0.0,32.0,

In [828]:
vec.printSchema()

root
 |-- Category: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex: long (nullable = true)
 |-- ALB: double (nullable = true)
 |-- ALP: double (nullable = true)
 |-- ALT: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: double (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: double (nullable = true)
 |-- vectors: vector (nullable = true)



In [829]:
#Podzial na zbior testowy i treningowy

In [830]:
train_df, test_df = vec.randomSplit([0.7,0.3])

In [831]:
test_df.count()

186

In [832]:
train_df.count()

429

In [833]:
#Zbudowanie modelu regresji logistycznej i drzewa decyzynego

In [834]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

In [835]:
lr = LogisticRegression(featuresCol='vectors', labelCol="Category")

In [836]:
model_lr = lr.fit(train_df)

In [837]:
predict_y = model_lr.transform(test_df)

In [838]:
#predict_y.show()

In [839]:
print(predict_y.columns)

['Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'vectors', 'rawPrediction', 'probability', 'prediction']


In [840]:
predict_y.select('Category','rawPrediction', 'probability', 'prediction').show()

+--------+--------------------+--------------------+----------+
|Category|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+----------+
|       0|[569.826313030913...|[1.0,0.0,9.784442...|       0.0|
|       0|[509.946862212935...|[1.0,9.1867475356...|       0.0|
|       0|[545.063264020359...|[1.0,0.0,8.736686...|       0.0|
|       0|[661.932529334860...|[1.0,0.0,3.562056...|       0.0|
|       0|[572.779961129525...|[1.0,7.0264065784...|       0.0|
|       0|[553.828637273270...|[1.0,0.0,5.986596...|       0.0|
|       0|[582.713651657958...|[1.0,0.0,9.944532...|       0.0|
|       0|[647.450835691861...|[1.0,0.0,1.545629...|       0.0|
|       0|[625.061340742442...|[1.0,0.0,2.496758...|       0.0|
|       0|[633.126259383484...|[1.0,0.0,1.134518...|       0.0|
|       0|[678.348953546221...|[1.0,0.0,2.653917...|       0.0|
|       0|[607.120201082369...|[1.0,0.0,4.358761...|       0.0|
|       0|[677.468400701193...|[1.0,0.0,

In [841]:
#Przed porownaniem zmieniam typ danych w kolumnie "Category" na double, bo inaczej nie porowna

In [842]:
pred_y = predict_y.withColumn('Category',col('Category').cast(DoubleType()))

#### Ocena modulu

In [843]:
# Dokladnosc

In [844]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [845]:
multiclass_eval = MulticlassClassificationEvaluator(labelCol='Category', metricName ='accuracy')

In [846]:
multiclass_eval.evaluate(pred_y)

0.9838709677419355

In [847]:
# Precyzja, F1, recall

In [848]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [849]:
# porownuje kategorie pacjenta z przewidziana, wybieram te kolumny ze zbioru przewidywanego predict_y

In [850]:
metrics_lr = MulticlassMetrics(pred_y['Category', 'prediction'].rdd)

In [851]:
#dir(metrics_lr)

In [852]:
print ("Dokladnosc", metrics_lr.accuracy)

Dokladnosc 0.9838709677419355


In [853]:
print ("Precyzja", metrics_lr.precision(0.0))

Precyzja 1.0


In [854]:
print ("Recall", metrics_lr.recall(0.0))

Recall 0.9939393939393939


In [855]:
print ("wspolczynnik F1", metrics_lr.fMeasure(0.0))

wspolczynnik F1 0.9969604863221885


In [ ]:
#Wniosek model predyckji jest idelany do tego zadania (patrz ocena), ale moze nie nadawac sie do np. predyckji innych chorob wartoby